In [15]:
from collections import deque
import MCTS
import numpy as np
from ConnectX import ConnectX
from Agent import Agent_random, Agent_DNN

episodes = 400
outcomes = []
losses = []

agent = Agent_DNN()

import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episodes).start()

for e in range(episodes):

    mytree = MCTS.Node(ConnectX())
    vterm = []
    logterm = []
    
    while mytree.winner is None:
        for _ in range(50):
            mytree.explore(agent)

        current_player = mytree.game.player
        mytree, (v, nn_v, p, nn_p) = mytree.next()        
        mytree.detach_mother()
        
        # compute prob* log pi 
        loglist = np.log(nn_p)*p
        
        # constant term to make sure if policy result = MCTS result, loss = 0
        constant = torch.where(p>0, p*torch.log(p),torch.tensor(0.))
        logterm.append(-torch.sum(loglist-constant))
        
        vterm.append(nn_v*current_player)

        
    # we compute the "policy_loss" for computing gradient
    outcome = mytree.outcome
    outcomes.append(outcome)
    
    loss = torch.sum( (torch.stack(vterm)-outcome)**2 + torch.stack(logterm) )
    optimizer.zero_grad()

    loss.backward()
    losses.append(float(loss))
    optimizer.step()
    
    if (e+1)%50==0:
        print("game: ",e+1, ", mean loss: {:3.2f}".format(np.mean(losses[-20:])),
              ", recent outcomes: ", outcomes[-10:])
    del loss
    
    timer.update(e+1)
    
    
timer.finish()

ValueError: max() arg is an empty sequence

In [1]:
import MCTS

from copy import copy
import random
from ConnectX import ConnectX

from Agent import Agent_random, Agent_DNN
agent = Agent_DNN()

def Agent_Player_MCTS(game):
    mytree = MCTS.Node(copy(game))
    for _ in range(50):
        mytree.explore(agent)
   
    mytreenext, (v, nn_v, p, nn_p) = mytree.next(temperature=0.1)

Using plaidml.keras.backend backend.


In [2]:
game = ConnectX()
print(game)
Agent_Player_MCTS(game)

TypeError: 'int' object is not iterable